In [3]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import GRU, Flatten, Dense, Conv1D, Dropout, LeakyReLU
from tensorflow.keras import Sequential
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os

%load_ext blackcellmagic

# %%black for formatting

2022-02-02 20:00:47.706177: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-02 20:00:47.706207: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
def generator(input_dim, feature_size, output_dim=1):
    model = Sequential()
    model.add(
        GRU(
            units=1024,
            return_sequences=True,
            input_shape=(input_dim, feature_size),
            recurrent_dropout=0.2,
        )
    )
    model.add(GRU(units=512, return_sequences=True, recurrent_dropout=0.2))
    model.add(GRU(units=256, return_sequences=False, recurrent_dropout=0.2))
    model.add(Dense(128))
    model.add(Dense(64))
    model.add(Dense(units=output_dim))
    return model


def discriminator(input_shape):
    model = tf.keras.Sequential()
    model.add(
        Conv1D(
            32,
            input_shape=input_shape,
            kernel_size=3,
            strides=2,
            padding="same",
            activation=LeakyReLU(alpha=0.01),
        )
    )
    model.add(
        Conv1D(
            64,
            kernel_size=5,
            strides=2,
            padding="same",
            activation=LeakyReLU(alpha=0.01),
        )
    )
    model.add(
        Conv1D(
            128,
            kernel_size=5,
            strides=2,
            padding="same",
            activation=LeakyReLU(alpha=0.01),
        )
    )
    model.add(Flatten())
    model.add(Dense(220, use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(220, use_bias=False, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [5]:
class Gan:
    def __init__(self, batch_size, learning_rate, generator, discriminator):
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.generator = generator
        self.generator_optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        self.discriminator = discriminator
        self.discriminator_optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        self.loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        self.checkpoint_directory = str(
            Path(os.path.abspath("").parents[0]) / "/checkpoints/"
        )
        self.checkpoint_prefix = str(Path(self.checkpoint_directory) / "ckpt")
        self.checkpoint = tf.train.Checkpoint(
            generator_optimizer=self.generator_optimizer,
            discriminator_optimizer=self.discriminator_optimizer,
            generator=self.generator,
            discriminator=self.discriminator,
        )

    def discriminator_loss(self, real_output, fake_output):
        real_loss = self.cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = self.cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    def generator_loss(self, fake_output):
        return self.cross_entropy(tf.ones_like(fake_output), fake_output)

    @tf.function
    def train_step(self, real_x, real_y): 
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            pass 

In [26]:
discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 2, 32)             128       
                                                                 
 conv1d_7 (Conv1D)           (None, 1, 64)             10304     
                                                                 
 conv1d_8 (Conv1D)           (None, 1, 128)            41088     
                                                                 
 flatten_1 (Flatten)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 220)               28160     
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 220)               0         
                                                                 
 dense_4 (Dense)             (None, 220)              